In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")


In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [5]:
documents


[Document(id_='cf4cd26b-67e7-49d0-914a-a1f0c7c6eff5', embedding=None, metadata={'page_label': '1', 'file_name': '/Users/swapnildhamu/Developer/vin-ai/RAG/llamaindex_and_openAI/data/Customer Master _ Vinculum Knowledge Central.pdf', 'file_path': '/Users/swapnildhamu/Developer/vin-ai/RAG/llamaindex_and_openAI/data/Customer Master _ Vinculum Knowledge Central.pdf', 'file_type': 'application/pdf', 'file_size': 456124, 'creation_date': '2024-03-11', 'last_modified_date': '2024-03-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='11/03/2024, 15:48 Customer Master : Vinculum Knowledge Central\nhttps://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000212663-customer-master 1/4Customer Master\nModiﬁed on: Wed, 31 Jan, 2024 at 6:03 PM\nCustomer Ma

In [6]:
index=VectorStoreIndex(documents, show_progress=True)

/Users/swapnildhamu/Developer/vin-ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 69/69 [00:02<00:00, 25.75it/s]


In [7]:
index

In [8]:
query_engine=index.as_query_engine() 

In [9]:
query_engine

In [17]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


retriever=VectorIndexRetriever(index=index)


postProcessor=SimilarityPostprocessor(similarity_cutoff=0.85)

query_engine=RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postProcessor])

In [12]:
response=query_engine.query("Want to add new SKU’s/ Items Please assist?")

In [13]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)

Final Response: To add new SKUs or items, you can use the "Add New"
option in the system. After clicking on this option, a screen will
open where you can choose the SKU classification from the dropdown
menu. The major classifications available are Gradable, Normal, Style,
and BOM. You can select the appropriate classification based on your
requirements and proceed with creating the new SKU or item.
______________________________________________________________________
Source Node 1/3
Node ID: 0599a56b-8ba2-48be-8903-0753318369a3
Similarity: 0.8314592697353788
Text: 03/03/2024, 12:05 SKU Master : Vinculum Knowledge Central https:
//vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000213659
-sku-master 3/7Action Buttons Description SearchClicking on the search
button without entering any filter criterion will populate the list of
all the SKUs created so far based on default filter setting. User can
f...
______________________________________________________________________
Sour

In [10]:
print(response)

To edit any SKU field, you need to navigate to the SKU Enquiry screen and click on the ‘Blue’ colored SKU code. This action will open the SKU Create/Edit screen, where you can make the necessary changes to the SKU details.


## Stroing the index

In [10]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
# response = query_engine.query("What are SKU?")
# print(response)
queries=[
    "What are SKU?",
    "Want to add new SKU's/ Items Please assist?",
    "How can we update the SKU details like SKU Name, Price, Description in Bulk?",
    "Getting Error as Color not available while uploading item master?",
    "Getting Error as UOM not available while uploading item master?",
    "Getting Error as size not available while uploading item master?",
    "Getting Error as Hierarchy not available while uploading item master?",
    "Getting Error as HSN Code not available while uploading item master?",
    "Getting Error as Brand not available while uploading item master?",
    "Getting Error as Upload SKU within assigned limit",
    "Getting error as “Excel Cell [D9] must not exceed the length above 50 characters” while uploading the SKU master",
    "Getting error as “Excel Cell [G9] must not exceed the length above 200 characters” while uploading the SKU master",
    "Getting Invalid Template Error",
    "Getting error as Duplicate SKU code while uploading item master",
    "Error due to wrong format.",
    "Server Encountered Error while uploading the File",
    "File uploaded successfully but showing under processing.",
    "Want to upload SKU Barcode / EAN code in the system against uploaded SKU’s",
    "Facing Error as EAN/barcode no is already mapped with SKU while uploading the SKU master.",
    "Getting an error as “This SKU is not a valid Sku” while uploading the SKU barcode."
]
# for query in queries:
#     response = query_engine.query(query)
#     print(f"user: {query}")
#     # pprint_response(response, show_source=True)
#     print(f"Assistant: {response}")
#     print()

In [71]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
    print("Created and stored index")
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
    print("Loaded index from storage")

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What is SKU Import module?")
print(response)

Loaded index from storage
The SKU Import module is a feature in Vin eRetail that allows users to import SKUs in bulk using a CSV template file. It consists of two sections: Import and Download. The Import section enables users to import SKUs in bulk by selecting the Import Type, Attribute Set, and uploading the correct CSV file. The Download section allows users to search and download the uploaded CSV file of SKU imports based on various criteria.


## Trading Partner FAQ’s:


In [24]:
# Can I pull my sku mapping through marketplace
# Need assistance in creating New Vendor?
# Getting Error as “Vendor code is Mandatory” while uploading Vendor master
# Getting Error as “Vendor Name is Mandatory” while uploading Vendor master
# Getting Error as “Tax Zone is Mandatory” while uploading Vendor master
# Getting Error as “Currency Code is Mandatory” while uploading Vendor master
# Facing Error as “Vendor type Mandatory Error”
# Want to add Customer master Please assist?
# Getting error as Customer type is Mandatory, how can this be eliminated?
# Why am I getting an error as GST no is mandatory while adding customer master?

# queries=[
#     "What is SKU Import module?",
#     "Need assistance in creating New Vendor?",
#     "Getting Error as “Vendor code is Mandatory” while uploading Vendor master",
#     "Getting Error as “Vendor Name is Mandatory” while uploading Vendor master",
#     "Getting Error as “Tax Zone is Mandatory” while uploading Vendor master",
#     "Getting Error as “Currency Code is Mandatory” while uploading Vendor master",
#     "Facing Error as “Vendor type Mandatory Error”",
#     "Want to add Customer master Please assist?",
#     "Getting error as Customer type is Mandatory, how can this be eliminated?",
#     "Why am I getting an error as GST no is mandatory while adding customer master?"
# ]

# queries=["can i manage multiple taxes in vinculum? if yes how can i do that?",]
queries=[""]

for query in queries:
    response = query_engine.query(query)
    print(f"user: {query}")
    # pprint_response(response, show_source=True)
    print(f"Assistant: {response}")
    print()

user: Mujhe customer master add karte samay GST number mandatory error kyun aa raha hai?
Assistant: Aapko customer master add karte samay GST number mandatory error tab aata hai jab aapne customer type ko B2B select kiya hai, kyunki B2B customers ke liye GST number mandatory hota hai. Isliye ya to aap customer type ko B2C mein change karein ya fir agar B2B hai to relevant GST number add karne ke saath proceed karein.

